## Setup

 Just to verify that everything's working.

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
from keras.models import Sequential
from keras.layers import Dense,LSTM,Activation,InputLayer,Input,BatchNormalization
from keras.callbacks import ModelCheckpoint
from sklearn.feature_extraction.text import CountVectorizer
# Verify we are using GPU.
from tensorflow.python.client import device_lib
import sys
print(sys.maxsize)
assert sys.version_info[0] >= 3
print("Set up!")

## Preprocessing

For the preprocessing step, we will create two dictionaries. One will be used to map questions to articles, while the other will be used to map questions and the contents of the articles to the answers.

### Loading JSON datasets

In [ ]:
p = re.compile('[1-9]*[1-9]')
def n2w(_string):
    isInt = True
    stringToReturn = ""
    try:
        stringToReturn = num2words(int(_string))
    except:
        stringToReturn = _string
        #assert isinstance(stringToReturn,str)
        return stringToReturn
def convertNumbersToWords(_string):
    #Error: expected string?
    #assert isinstance(_string,str)
    _string_copy = p.sub(lambda x: n2w(x.group()),_string)
    return _string_copy

In [2]:
import json
import re
import string
import pandas as pd
def readFile(filename):
    with open(filename) as file:
        data = json.load(file)["data"]
        paragraphs = pd.io.json.json_normalize(data,record_path="paragraphs")
        # Map paragraph IDs and qas IDs.
        paragraphIDs_questions = []
        lastParagraphId = 0
        for index,paragraph in paragraphs.iterrows():
            for qasElement in paragraph["qas"]:
                paragraphIDs_questions.append({"paragraphID":lastParagraphId,"question":qasElement["question"]})
            lastParagraphId = lastParagraphId + 1
        paragraphIDs_questions = pd.DataFrame(paragraphIDs_questions)
        print("Finished mapping paragraph IDs to qas questions. Now generating qas dataframe and doing other things.")
        qas = pd.io.json.json_normalize(data,record_path=["paragraphs","qas"],meta=["title"])
        paragraphs["id"] = paragraphs.index
        #print(qas["question"])
        #Gather a list of where all answers should be so we can shove them into a DataFrame.
        # Haven't found a more efficient way to do this yet.
        answer_ids = set()
        answerId = 0
        for index,row in qas.iterrows():
            answer_ids.add(answerId)
            answerId = answerId + len(row["answers"])
        print("Finished with answer ids.")
        # Map qas pair IDs to answer IDs.
        answer_ids = pd.DataFrame(list(answer_ids))
        print("Finished converting answer_ids to DataFrame.")
        question_answerId = pd.DataFrame(qas["question"]).join(answer_ids,how="outer")
        question_answerId.columns = ["question","answer_id"]
        #print("Id-answerID columns: ",id_answerId.columns)
        print("finished creating intermediary table.")
        # Load answers into a data frame.
        answers = pd.io.json.json_normalize(data,record_path=["paragraphs","qas","answers"])
        answers.rename(columns={"text":"answer_text"},inplace=True)
        # Give each answer an ID.
        answers["id"] = answers.index
        print("Finished creating answers dataframe.")
        qas = qas.drop(labels=["answers"],axis=1) # Not needed any longer; we have the answers!
        #print("Dropped column 'answers' from qas.")
        # Map qas dataframe to answer table via id_answerId
        qas_answerId = pd.merge(qas,question_answerId,how="inner",on="question")
        # Check that no duplicates exist in qas_answerId
        qas_answerId = qas_answerId.drop_duplicates("question")
        assert qas_answerId.duplicated("question").any() == False
        print("Finished joining qas to answer id")
        # Merge qas_answerId with answers.
        qas = pd.merge(qas_answerId,answers,how="inner",left_on="answer_id",right_on="id")
        #print("Returned data frame: ",returnDataFrame)
        # Finally, include context.
        context = paragraphs[["context","id"]]
        qasWithParagraphID = pd.merge(qas,paragraphIDs_questions,how="inner",on="question")
        qas = pd.merge(qasWithParagraphID,context,how="inner",left_on="paragraphID",right_on="id")
        print("Done!")
        return qas

In [3]:
# Test case.
dataFrame = readFile("train-v1.1.json")
print (dataFrame.loc[dataFrame["question"] == "When did Beyoncé release her first solo album?"])

Finished mapping paragraph IDs to qas questions. Now generating qas dataframe and doing other things.


KeyboardInterrupt: 

In [4]:
# Test that the above code works.
import json
dataFrame = readFile("train-v1.1.json")
#print(dataFrame)
with open("train-v1.1.json") as file:
    data = json.load(file)["data"]
    for article in data:
        for paragraph in article["paragraphs"]:
            paragraph_context = paragraph["context"]
            qas = paragraph["qas"]
            dataFrameContext = dataFrame[dataFrame.context == paragraph_context]
            assert dataFrameContext in paragraph_context, "Context not matching!"
            for _qas in qas:
                question = _qas["question"]
                answers = [answer["text"] for answer in _qas["answers"]]
                dataFrameAnswers = dataFrame[dataFrame.question == question]["answer_text"].values
                # If the above function works, the dataframe should contain the same question/answer pair.
                assert dataFrameAnswers.shape[0] == 1, "Assertion failed. Question:" + question + " answers: " + dataFrameAnswers
                #message = "data frame answered " + dataFrameAnswers + "; actual answer is " + answer + " question: " + question
                assert type(answers[0]) == type(dataFrameAnswers[0])
                assert str(dataFrameAnswers[0]) in answers, "dataframe answer: " + str(dataFrameAnswers) + " actual answers:" + str(answers)
print("All good!")

Finished mapping paragraph IDs to qas questions. Now generating qas dataframe and doing other things.
Finished with answer ids.
Finished converting answer_ids to DataFrame.
finished creating intermediary table.
Finished creating answers dataframe.
Finished joining qas to answer id
Done!


TypeError: 'in <string>' requires string as left operand, not DataFrame

In [ ]:
trainingData = readFile("train-v1.1.json")
print(trainingData)
from sys import getsizeof
print("Finished loading training data.")
print("Size of training data:",getsizeof(trainingData))

In [ ]:
print("Amount of training data: ",len(training_questions))

In [ ]:
dev_questions,dev_articleTitles,dev_articleTexts,dev_answers = readFile("dev-v1.1.json")
print("Finished loading dev data.")
print("Size of dev questions",getsizeof(dev_questions) / 1024)
print("Size of dev titles",getsizeof(dev_articleTitles) / 1024)
print("Size of dev texts",getsizeof(dev_articleTexts) / 1024)
print("Size of dev answers",getsizeof(dev_answers) / 1024)

In [ ]:
assert len(dev_questions) == len(dev_answers)
print("Amount of dev data: " + str(len(dev_questions)))

### Text preprocessing using scikit-learn and TensorFlow

### Get n-grams

The following section turns inputs into n-grams. Note that order is not necessarily preserved.

In [ ]:
#Custom skip-gram vectorizer
from toolz import itertoolz, compose
from toolz.curried import map as cmap, sliding_window, pluck
from sklearn.feature_extraction.text import CountVectorizer

class SkipGramVectorizer(CountVectorizer):
    def build_analyzer(self):    
        preprocess = self.build_preprocessor()
        stop_words = self.get_stop_words()
        tokenize = self.build_tokenizer()
        return lambda doc: self._word_skip_grams(
                compose(tokenize, preprocess, self.decode)(doc),
                stop_words)
    
    def _word_skip_grams(self, tokens, stop_words=None):
        """Turn tokens into a sequence of 1-skip-2-grams after stop words filtering"""
        # handle stop words
        if stop_words is not None:
            tokens = [w for w in tokens if w not in stop_words]

        return compose(cmap(' '.join), pluck([0, 2]), sliding_window(3))(tokens)

#### Vectorize questions

In [ ]:
def vectorize(trainData,devData):
    vectorizer = SkipGramVectorizer()
    vectorizer.fit(trainData + devData)
    print("Vectorizer fit.")
    trainData_vectorized = vectorizer.transform(trainData)
    print("First text vectorized")
    devData_vectorized = vectorizer.transform(devData)
    print("Second text vectorized")
    return trainData_vectorized,devData_vectorized,vectorizer.vocabulary_

In [ ]:

# Vectorize all questions.
training_questions,dev_questions,question_vocabulary = vectorize(training_questions,dev_questions)
print("Number of features:",training_questions.shape)

#### Vectorize article titles

In [ ]:
# Training set.
training_articleTitles,dev_articleTitles,articleTitles_vocabulary = vectorize(training_articleTitles,dev_articleTitles)
print("Article titles vectorized")

#### Vectorize article content

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
contentBinarizer = MultiLabelBinarizer()
print(getsizeof(contentBinarizer))
contentTotal = training_articleTexts + dev_articleTexts
print("Gathered content")
contentBinarizer.fit(contentTotal)
print("Content binarizer fit.")

In [ ]:
training_answers = contentBinarizer.transform(training_answers)
print("Training answers binarized.")

In [ ]:
dev_answers = contentBinarizer.transform(dev_answers)
print("Dev answers binarized")

In [ ]:
print("Number of features in questions:",training_questions.shape[1])
print("Number of features in answers:",training_answers.shape[1])
print("Number of features in article titles:",training_articleTitles.shape[1])
print("Number of features in content:",training_articleContent.shape[1])

## Data normalization

### More setup

In [ ]:
# convert data to numpy arrays, get input shape
inputShape_first = training_questions.toarray().shape[1:] # Input shape of the first neural network.
article_titles_shape = training_articleTitles.toarray().shape
print("First neural network X shape: " + str(training_questions.shape))
print("First neural network input shape: " + str(inputShape_first))
print("Article titles array shape: " + str(article_titles_shape))

#### Splitting the data into testing and cross-validation sets.


In [ ]:
# 20% of data will be set aside for cross-validation
from sklearn.model_selection import train_test_split
X_1_train,X_1_cross_validation,Y_1_train,Y_1_cross_validation = train_test_split(training_questions,training_articleTitles)
print(type(X_1_train))
print(X_1_train.shape)
print(Y_1_train.shape)
print(X_1_cross_validation.shape)
print(Y_1_cross_validation.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack
X_2 = hstack((training_questions,training_articleTitles))
print("X 2 horizontally stacked.")
X_2_train,X_2_cross_validation,Y_2_train,Y_2_cross_validation = train_test_split(X_2,training_answers)

In [ ]:
X_2_dev = hstack((dev_questions,dev_articleTitles))
print("X 2 dev horizontally stacked.")

#### Save preprocessed datasets to a file.

In [ ]:
from scipy.sparse import save_npz
save_npz("X_1_train",X_1_train)
save_npz("X_1_cross_validation",X_1_cross_validation)
save_npz("dev_questions",dev_questions)
save_npz("X_2_train",X_2_train)
save_npz("X_2_cross_validation",X_2_cross_validation)
save_npz("X_2_dev",X_2_dev)

In [ ]:
from scipy.sparse import save_npz
save_npz("Y_1_train",Y_1_train)
save_npz("Y_1_cross_validation",Y_1_cross_validation)
save_npz("dev_articleTitles",dev_articleTitles)
save_npz("Y_2_train",csr_matrix(Y_2_train))
save_npz("Y_2_cross_validation",csr_matrix(Y_2_cross_validation))
save_npz("dev_answers",csr_matrix(dev_answers))
print("Done!")

## Post-preprocessing analysis

In [ ]:
# Get vocabularies from preprocessors above.


#### N-gram pyplot